# 검색 결과를 기반으로 답변 생성하기

* 검색된 문서와 질문을 결합하여 GPT에게 전달하고,
* 실제로 문서 기반 답변을 생성해보는 **RAG의 핵심 구조**를 실습합니다.

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

### 1. 인덱스 및 모델 로딩

In [9]:
# E5 임베딩 모델
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")

# 저장된 FAISS 인덱스 로드
faiss_index = FAISS.load_local(
    "embeddings/faiss_index_e5",
    embeddings=embedding,
    allow_dangerous_deserialization=True
)

# 검색기 생성
retriever = faiss_index.as_retriever(search_kwargs={"k": 3})

# OpenAI LLM (gpt-3.5-turbo 사용)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### 2. RetrievalQA 체인 구성 (Stuff 방식)

- Stuff 방식은 **검색된 문서를 그대로 묶어서 LLM에 넣는 단순하고 빠른 방식**입니다.
- 짧은 문서에는 적합하지만, 문서가 길면 **토큰 초과**로 실패할 수 있습니다.

In [10]:
# Retrieval + LLM을 결합한 QA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"  # 문서를 한 번에 묶어서 전달
)

# 테스트 질문
query = "대한민국 국회의원의 임기는 몇 년인가요?"
answer = qa_chain.run(query)
print("질문:", query)
print("답변:", answer)

질문: 대한민국 국회의원의 임기는 몇 년인가요?
답변: 대한민국 국회의원의 임기는 4년입니다.


### 3. 다양한 Prompt 구성 방식 (개념 정리)

LangChain에서는 다음과 같은 문서 전달 방식(Chain Type)을 선택할 수 있습니다.

| 방식 | 설명 | 특징 |
|------|------|------|
| `stuff` | 모든 문서를 한 번에 넣음 | 빠르지만 토큰 제한 있음 |
| `map_reduce` | 문서별로 LLM 호출 후 요약 | 느리지만 안정적 |
| `refine` | 초기 응답 후 문서를 추가로 반영 | 대화형 요약에 적합 |
| `map_rerank` | 각 문서별로 점수 매기고 최고 답변 선택 | 정확도 중시 |

기본적으로 `stuff`부터 실습하고, 이후 프로젝트에 따라 `refine` 또는 `map_reduce`도 고려해볼 수 있습니다.

### 4. RAG 흐름 정리
전체 흐름은 다음과 같습니다:

1. 사용자 질문 → 임베딩 벡터 생성
2. 벡터 DB에서 유사 문서 검색 (FAISS)
3. 검색된 문서와 질문을 GPT에게 함께 전달
4. LLM이 문서를 참고하여 최종 답변 생성

이게 바로 Retrieval-Augmented Generation (RAG) 구조입니다.

### 5. 응용 실습

In [11]:
queries = [
    "대통령은 몇 년마다 선출되나요?",
    "대한민국의 수도는 어디인가요?",
    "헌법 제1조는 어떤 내용을 담고 있나요?"
]

for q in queries:
    print(f"\n[Q] {q}")
    print("[A]", qa_chain.run(q))


[Q] 대통령은 몇 년마다 선출되나요?
[A] 대통령은 5년마다 국민의 보통ㆍ평등ㆍ직접ㆍ비밀선거에 의하여 선출됩니다.

[Q] 대한민국의 수도는 어디인가요?
[A] 대한민국의 수도는 서울입니다.

[Q] 헌법 제1조는 어떤 내용을 담고 있나요?
[A] 헌법 제1조는 "이 헌법은 1988년 2월 25일부터 시행한다. 다만, 이 헌법을 시행하기 위하여 필요한 법률의 제정ㆍ개정과 이 헌법에 의한 대통령 및 국회의원의 선거 기타 이 헌법시행에 관한 준비는 이 헌법시행 전에 할 수 있다." 라는 내용을 담고 있습니다.


### 8. 정리 및 다음 단계

* 검색된 문서와 질문을 함께 GPT에게 전달하면, **정확도와 신뢰도가 높아진 답변**을 얻을 수 있습니다.
* Prompt 구성 방식에 따라 RAG의 응답 품질이 달라질 수 있습니다.
* 다음 단계에서는 **성능을 높이기 위한 최적화 방법 (Chunking, Hybrid Search 등)** 을 배웁니다.